[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JLrumberger/TensorflowTryOuts/blob/master/1%20CNN%20with%20Tensorflow.ipynb)
## CNN with Tensorflow
We first load the dataset and construct the placeholders for the input data and targets.

In [0]:
import tensorflow as tf
import os
#from tensorflow.examples.tutorials.mnist import input_data
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data(path='mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = tf.one_hot(y_train, depth=10), tf.one_hot(y_test, depth=10)


n_samples = 60000
n_classes = 10 # MNIST total classes (0-9 digits)

11493376/11490434 [==============================] - 0s 0us/step


## Prepare the dataset
`Dataset.from_tensor_slices` accepts one argument that can consist of seveal tf.tensors oder np.ndarrays. It can give out slices of the dataset (eg. batches), whereas `Dataset.from_tensors` can only give out the whole dataset. `Dataset.from_generator` lets you create the dataset at runtime via a generator function. This is useful for huge datasets that don't fit on your harddrive.

`Dataset.batch(int)` is used to split the dataset into batches of size `int`. 

`Dataset.repeat(int)` duplicates and concatenates the duplicate with the existing dataset `int` times. Without an argument, this function just repeats the dataset as often as needed for the evaluation of a graph. 

`Dataset.map(fn)` applies the function `fn` on all elements of the dataset elementwise. 

`Dataset.filter(cond)` lets you filter the dataset based on condition `cond`, 

`Dataset.zip()` allows you to zip together different Dataset objects, just like the python zip function.

More on this topic: [tensorflow-dataset-tutorial](http://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/)

In [0]:
# set up a Dataset object for the train data
data_train = tf.data.Dataset.from_tensor_slices((tf.cast(x_train, tf.float32), y_train))
data_train = data_train.repeat().batch(200)
# set up a Dataset object for the test data
data_test = tf.data.Dataset.from_tensor_slices((tf.cast(x_test, tf.float32), y_test))
data_test = data_test.repeat().batch(200)
# create a general iterator for the datasets and the get_next method
iterator = tf.data.Iterator.from_structure(data_train.output_types,
                                           data_train.output_shapes)
iterator_test = tf.data.Iterator.from_structure(data_test.output_types,
                                           data_test.output_shapes)
next_element = iterator.get_next()
next_element_test = iterator_test.get_next()
# create Iterator initiliazers for both Datasets
training_init_op = iterator.make_initializer(data_train)
test_init_op = iterator.make_initializer(data_test)


Next we wrap the tensorflow `conv2d` and `max_pool` functions, the convolution kernel size is defined by the shape of $W$, the pooling kernel size has to be declared.

In [0]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

## Build the model
Next we build the CNN model. First we declare the variables, then the computational graph.

In [0]:
n_nodes_hl1 = 32
n_nodes_hl2 = 64
n_nodes_hl3 = 1024
n_classes = 10
batch_size = 200


def convolutional_neural_network(x):
    weights = {
        # 5 x 5 convolution, 1 input image, 32 outputs
        'W_conv1': tf.Variable(tf.random_normal([5, 5, 1, n_nodes_hl1])),
        # 5x5 conv, 32 inputs, 64 outputs 
        'W_conv2': tf.Variable(tf.random_normal([5, 5, n_nodes_hl1, n_nodes_hl2])),
        # fully connected, 7*7*64 inputs, 1024 outputs
        'W_fc': tf.Variable(tf.random_normal([7*7*n_nodes_hl2, n_nodes_hl3])),
        # 1024 inputs, 10 outputs (class prediction)
        'out': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes]))
    }

    biases = {
        'b_conv1': tf.Variable(tf.random_normal([n_nodes_hl1])),
        'b_conv2': tf.Variable(tf.random_normal([n_nodes_hl2])),
        'b_fc': tf.Variable(tf.random_normal([n_nodes_hl3])),
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    # Reshape input to a 4D tensor 
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # Convolution Layer, using our function
    x = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    # Max Pooling (down-sampling)
    x = maxpool2d(x)
    # Convolution Layer
    x = tf.nn.relu(conv2d(x, weights['W_conv2']) + biases['b_conv2'])
    # Max Pooling (down-sampling)
    x = maxpool2d(x)
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer
    x = tf.reshape(x, [-1, 7*7*n_nodes_hl2])
    x = tf.nn.relu(tf.matmul(x, weights['W_fc']) + biases['b_fc'])
    output = tf.matmul(x, weights['out']) + biases['out']
    return output

Next: write a training function:

In [0]:
def train_neural_network():
    # declare all important functions and the model graph first
    prediction = convolutional_neural_network(next_element[0])
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=next_element[1]))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    equality = tf.equal(tf.to_float(tf.argmax(prediction,1)), tf.to_float(tf.argmax(next_element[1], 1)))    
    accuracy = tf.reduce_mean(tf.to_float(equality))
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(training_init_op)
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(n_samples/batch_size)):
                _, c = sess.run([optimizer, cost])
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)
        # initialize test data as data source and compute the accuracy
        sess.run(test_init_op)
        avg_acc = 0
        for i in range(100):
          acc = sess.run([accuracy])
          avg_acc +=acc[0]
        print('Accuracy:',avg_acc)
train_neural_network()

Epoch 0 completed out of 10 loss: 983986.0851287842
Epoch 1 completed out of 10 loss: 206543.25619506836
Epoch 2 completed out of 10 loss: 126760.17093205452
Epoch 3 completed out of 10 loss: 89780.49850058556
Epoch 4 completed out of 10 loss: 63913.13455379009
Epoch 5 completed out of 10 loss: 50297.55650415834
Epoch 6 completed out of 10 loss: 38080.19561898708
Epoch 7 completed out of 10 loss: 29794.981118291616
Epoch 8 completed out of 10 loss: 26291.624722008706
Epoch 9 completed out of 10 loss: 19571.69092036411
Accuracy: 96.72000002861023


## ResNets

### Residual Block
We implement the right one
![grafik.png](https://i.stack.imgur.com/lFNWB.png)

In [0]:
def resblock(x, filters, is_training):
    x = tf.layers.batch_normalization(x,training=is_training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x,filters,(3,3),padding='same')
    x = tf.layers.batch_normalization(x,training=is_training)
    x = tf.nn.relu(x)
    x = tf.layers.conv2d(x,filters,(3,3),padding='same')
    return x

In [0]:
def ResNet(x):
    # declare placeholder to indicate if it's training or testing
    is_training = tf.placeholder(tf.bool, name='is_training')
    filters = 64
    # Reshape input to a 4D tensor 
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    x = tf.layers.conv2d(x, filters=filters, kernel_size=(3,3),padding='same')
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = maxpool2d(x)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = maxpool2d(x)
    x += resblock(x,filters,is_training)
    x += resblock(x,filters,is_training)
    x = tf.reshape(x, [-1, 7*7*filters])
    x = tf.layers.dense(x,512,activation='relu')
    out = tf.layers.dense(x,10,activation='softmax')
    return out
    

In [0]:
def train_neural_network():
    # declare some variables
    n_classes = 10
    batch_size = 200
    hm_epochs = 10
    # declare all important functions and the model graph
    prediction = ResNet(next_element[0])
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=next_element[1]))
    equality = tf.equal(tf.to_float(tf.argmax(prediction,1)), tf.to_float(tf.argmax(next_element[1], 1)))    
    accuracy = tf.reduce_mean(tf.to_float(equality))
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    # make sure to update_ops when running the optimizer, otherwise the population moments
    # inside batch_norm are not calculated correctly and BN doesn't work
    with tf.control_dependencies(update_ops):
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)
    with tf.Session() as sess:
        # initialize variables and interator
        sess.run([tf.global_variables_initializer(),training_init_op])
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(n_samples/batch_size)):
                _, c = sess.run([optimizer, cost],feed_dict={'is_training:0':True})
                epoch_loss += c

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
        sess.run(test_init_op) 
        avg_acc = 0
        for i in range(100):
          acc = sess.run([accuracy],feed_dict={'is_training:0':False})
          avg_acc +=acc[0]
        print('Accuracy:',avg_acc)

train_neural_network()

Epoch 1 completed out of 10 loss: 513.4809859991074
Epoch 2 completed out of 10 loss: 454.109428524971
Epoch 3 completed out of 10 loss: 441.7268034219742
Epoch 4 completed out of 10 loss: 440.61018764972687
Epoch 5 completed out of 10 loss: 440.0227122306824
Epoch 6 completed out of 10 loss: 439.68425619602203
Epoch 7 completed out of 10 loss: 439.549938082695
Epoch 8 completed out of 10 loss: 439.5607305765152
Epoch 9 completed out of 10 loss: 439.3178821802139
Epoch 10 completed out of 10 loss: 439.31312918663025
Accuracy: 98.83000075817108
